In [ ]:
# for google colaboratory
from os import pathimportfrom opt_einsum.backends import torch torch
if 'google.colab' in str(get_ipython()):
    files = ['LLD-icon-names.pkl', 'LLD-icon_PKL.zip', 'LLD-icon-sharp.hdf5', 
             'LLD-icon-sharp_indices.pkl']
    !pip install wget
    import wget
    import shutil
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir Data
    for f in files:
        if not path.isfile('Data/' + f):
            if path.isfile('/content/drive/My Drive/Colab/AFRO/' + f):
                shutil.copy('/content/drive/My Drive/Colab/AFRO/' + f, 'Data')
            else:
                wget.download('https://data.vision.ee.ethz.ch/sagea/lld/data/' + f, 'Data')
    !unzip -q -n Data/LLD-icon_PKL.zip -d Data

In [ ]:
# for debugging ------
from importlib import reload
# -------------------
import pickle
import re

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from transformers import BertTokenizer, BertModel
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import notebook

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as sw


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Open data

## icon-sharp

In [ ]:
files = h5py.File('Data/LLD-icon-sharp.hdf5', 'r')

In [ ]:
files.keys()

In [ ]:
clusters = files['labels/resnet/rc_128']
names = files['meta_data/names']
images = files['data']

### show clasters

In [ ]:
unique = len(set(clusters[()]))
samples = 8
fig, axs = plt.subplots(unique, samples, figsize=(10, 50))
for i in range(unique):
        indices = np.argwhere(clusters[()]==i)
        for j in range(8):
                axs[i,j].imshow(np.moveaxis(images[indices[j]], 1, -1).squeeze(0))
                axs[i,j].axis('off')

## icon_data

In [ ]:
with open('Data/LLD-icon/LLD-icon_data_0.pkl', 'rb') as f:
    data = pickle.load(f, encoding='latin1')

In [ ]:
data.shape

In [ ]:
plt.imshow(data[1])

In [ ]:
with open('Data/LLD-icon-names.pkl', 'rb') as f:
    labels = pickle.load(f)

In [ ]:
len(labels)

# Create text and add to icon

In [ ]:
import albumentations as A
import cv2 as cv
# custom module --- 
import utils
utils = reload(utils)
from utils import text_to_pic_transform, join_images
# ------------------

In [ ]:
from skimage.transform import resize
size = (128, 128)
img = resize(np.moveaxis(images[0], 0, -1), size, mode='constant', anti_aliasing=True,)


In [ ]:
plt.imshow(img)

In [ ]:
text_img = text_to_pic_transform('Hello')


In [ ]:
plt.imshow(text_img)

In [ ]:
plt.imshow(join_images(img, text_img))

# Подготовка данных

In [ ]:
import models
models = reload(models)
from models import Generator, Discriminator, WGAN

import tensorflow.keras as keras
import tensorflow as tf

In [ ]:
plt.imshow(dst)

files = h5py.File('Data/LLD-icon-sharp.hdf5', 'r')
images = files['data']

batch_size = 64
images = np.array(images)
images = np.moveaxis(images, 1, -1)
images = images.astype("float32") / 255.0
dataset = tf.data.Dataset.from_tensor_slices(images)

# Создание и тренировка модели

In [ ]:
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)


In [ ]:
gen = Generator(100, 3)
disc = Discriminator((32, 32, 3))

In [ ]:
gan = WGAN(discriminator=disc, generator=gen, latent_dim=100)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

In [ ]:
gan.fit(dataset.take(100), epochs=1)